In [ ]:
#!pip install -U seaborn aws_requests_auth imblearn 

## We will download the Credit Fraud Data and store it in a feature store and make this feature store offline the main purpose of the feature store is to provide cinsistent data for different team members. We can also enable online feature store if we need to access the data in realtime

In [1]:
import boto3
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
from sagemaker import get_execution_role
role = get_execution_role()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Get Fraud Data

In [2]:
%%bash
wget https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
unzip creditcardfraud.zip

--2024-01-08 18:13:46--  https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.130.200, 52.92.146.0, 52.92.241.152, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.130.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip.1’

     0K .......... .......... .......... .......... ..........  0%  815K 83s
    50K .......... .......... .......... .......... ..........  0%  819K 83s
   100K .......... .......... .......... .......... ..........  0%  828K 82s
   150K .......... .......... .......... .......... ..........  0% 76.9M 62s
   200K .......... .......... .......... .......... ..........  0%  833K 66s
   250K .......... .......... .......... .......... ..........  0%  148M 55s
   300K .......... .......... .......... .......... ..........  0% 85.3M 47

  5050K .......... .......... .......... .......... ..........  7%  258M 6s
  5100K .......... .......... .......... .......... ..........  7%  252M 6s
  5150K .......... .......... .......... .......... ..........  7%  207M 6s
  5200K .......... .......... .......... .......... ..........  7%  264M 6s
  5250K .......... .......... .......... .......... ..........  7%  251M 6s
  5300K .......... .......... .......... .......... ..........  7%  247M 6s
  5350K .......... .......... .......... .......... ..........  7%  212M 6s
  5400K .......... .......... .......... .......... ..........  8%  254M 6s
  5450K .......... .......... .......... .......... ..........  8%  243M 6s
  5500K .......... .......... .......... .......... ..........  8%  245M 6s
  5550K .......... .......... .......... .......... ..........  8%  179M 5s
  5600K .......... .......... .......... .......... ..........  8%  240M 5s
  5650K .......... .......... .......... .......... ..........  8%  248M 5s
  5700K ....

 10450K .......... .......... .......... .......... .......... 15%  254M 3s
 10500K .......... .......... .......... .......... .......... 15%  250M 3s
 10550K .......... .......... .......... .......... .......... 15%  982K 4s
 10600K .......... .......... .......... .......... .......... 15%  151M 4s
 10650K .......... .......... .......... .......... .......... 15%  147M 4s
 10700K .......... .......... .......... .......... .......... 15%  172M 4s
 10750K .......... .......... .......... .......... .......... 15% 97.9M 4s
 10800K .......... .......... .......... .......... .......... 16%  112M 4s
 10850K .......... .......... .......... .......... .......... 16%  131M 3s
 10900K .......... .......... .......... .......... .......... 16%  135M 3s
 10950K .......... .......... .......... .......... .......... 16%  168M 3s
 11000K .......... .......... .......... .......... .......... 16%  149M 3s
 11050K .......... .......... .......... .......... .......... 16%  204M 3s
 11100K ....

 15850K .......... .......... .......... .......... .......... 23%  211M 3s
 15900K .......... .......... .......... .......... .......... 23%  309M 3s
 15950K .......... .......... .......... .......... .......... 23%  159M 3s
 16000K .......... .......... .......... .......... .......... 23%  255M 3s
 16050K .......... .......... .......... .......... .......... 23%  254M 3s
 16100K .......... .......... .......... .......... .......... 23%  209M 3s
 16150K .......... .......... .......... .......... .......... 23%  293M 3s
 16200K .......... .......... .......... .......... .......... 24%  353M 3s
 16250K .......... .......... .......... .......... .......... 24%  316M 3s
 16300K .......... .......... .......... .......... .......... 24%  250M 3s
 16350K .......... .......... .......... .......... .......... 24%  287M 2s
 16400K .......... .......... .......... .......... .......... 24%  336M 2s
 16450K .......... .......... .......... .......... .......... 24%  314M 2s
 16500K ....

 21250K .......... .......... .......... .......... .......... 31%  247M 2s
 21300K .......... .......... .......... .......... .......... 31%  251M 2s
 21350K .......... .......... .......... .......... .......... 31%  229M 2s
 21400K .......... .......... .......... .......... .......... 31%  253M 2s
 21450K .......... .......... .......... .......... .......... 31%  254M 2s
 21500K .......... .......... .......... .......... .......... 31%  224M 2s
 21550K .......... .......... .......... .......... .......... 31%  213M 2s
 21600K .......... .......... .......... .......... .......... 32%  223M 2s
 21650K .......... .......... .......... .......... .......... 32%  239M 2s
 21700K .......... .......... .......... .......... .......... 32%  224M 2s
 21750K .......... .......... .......... .......... .......... 32%  199M 2s
 21800K .......... .......... .......... .......... .......... 32% 1014K 2s
 21850K .......... .......... .......... .......... .......... 32%  181M 2s
 21900K ....

...

 59050K .......... .......... .......... .......... .......... 87%  191M 0s
 59100K .......... .......... .......... .......... .......... 87%  250M 0s
 59150K .......... .......... .......... .......... .......... 87%  212M 0s
 59200K .......... .......... .......... .......... .......... 87%  247M 0s
 59250K .......... .......... .......... .......... .......... 87%  254M 0s
 59300K .......... .......... .......... .......... .......... 87%  244M 0s
 59350K .......... .......... .......... .......... .......... 87%  218M 0s
 59400K .......... .......... .......... .......... .......... 88%  252M 0s
 59450K .......... .......... .......... .......... .......... 88%  250M 0s
 59500K .......... .......... .......... .......... .......... 88%  251M 0s
 59550K .......... .......... .......... .......... .......... 88%  188M 0s
 59600K .......... .......... .......... .......... .......... 88%  230M 0s
 59650K .......... .......... .......... .......... .......... 88%  218M 0s
 59700K ....

 64450K .......... .......... .......... .......... .......... 95%  252M 0s
 64500K .......... .......... .......... .......... .......... 95%  225M 0s
 64550K .......... .......... .......... .......... .......... 95%  222M 0s
 64600K .......... .......... .......... .......... .......... 95%  266M 0s
 64650K .......... .......... .......... .......... .......... 95%  255M 0s
 64700K .......... .......... .......... .......... .......... 95%  246M 0s
 64750K .......... .......... .......... .......... .......... 95%  203M 0s
 64800K .......... .......... .......... .......... .......... 96% 1.00M 0s
 64850K .......... .......... .......... .......... .......... 96%  176M 0s
 64900K .......... .......... .......... .......... .......... 96%  147M 0s
 64950K .......... .......... .......... .......... .......... 96%  127M 0s
 65000K .......... .......... .......... .......... .......... 96%  179M 0s
 65050K .......... .......... .......... .......... .......... 96%  174M 0s
 65100K ....

Archive:  creditcardfraud.zip


replace creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


CalledProcessError: Command 'b'wget https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip\nunzip creditcardfraud.zip\n'' returned non-zero exit status 1.

In [4]:
data = pd.read_csv('creditcard.csv', delimiter=',')

In [5]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
data.reset_index(inplace=True)

In [7]:
from datetime import datetime
# Set current time in the ISO-8601 format
current_time = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
data['event_time'] = current_time

In [8]:
data.head()

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,event_time
0,0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,2024-01-08T18:14:05Z
1,1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,2024-01-08T18:14:05Z
2,2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,2024-01-08T18:14:05Z
3,3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,2024-01-08T18:14:05Z
4,4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,2024-01-08T18:14:05Z


In [9]:
nonfrauds, frauds = data.groupby('Class').size()
pct_nonfrauds=100.*nonfrauds/(frauds + nonfrauds)
pct_frauds=100-pct_nonfrauds
print('Dataset size frauds: ', len(data))
print('Number of frauds: ', frauds)
print('Number of non-frauds: ', nonfrauds)
print('Percentage of Non fradulent data:', pct_nonfrauds)
print('Percentage of fradulent data:', pct_frauds)

Dataset size frauds:  284807
Number of frauds:  492
Number of non-frauds:  284315
Percentage of Non fradulent data: 99.827251436938
Percentage of fradulent data: 0.17274856306200093


## Preprocessing the data and store it in feature store

In [11]:
import sagemaker
import boto3
import pandas as pd
from sagemaker.session import Session

region = boto3.Session().region_name

# Initialize SageMaker and Boto3 clients
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')
featurestore_runtime = sagemaker_session.sagemaker_featurestore_runtime_client



feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Just using default bucket you can change it
default_s3_bucket_name = feature_store_session.default_bucket()
print(default_s3_bucket_name)

NameError: name 'feature_store_session' is not defined

In [13]:
from sagemaker.feature_store.feature_group import FeatureGroup

# Assuming you have a pandas DataFrame named 'df'
feature_group_name = 'fraud-feature-group'

feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

# Automatically infer the feature definitions from the DataFrame
feature_group.load_feature_definitions(data_frame=data)

[FeatureDefinition(feature_name='index', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='Time', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='V1', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='V2', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='V3', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='V4', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='V5', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='V6', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='V7', feature_type=<Fea

In [15]:
## delete feature store if it already exists
#sagemaker_client.delete_feature_group(FeatureGroupName=feature_group_name)

In [18]:
record_identifier_name = 'index'  
event_time_feature_name = 'event_time'

feature_group.create(
    s3_uri=f's3://sagemaker-us-east-1-470086202700/{feature_group_name}',  # Specify your S3 bucket and path
    record_identifier_name=record_identifier_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,  # Replace with your IAM role ARN
    enable_online_store=False  # Set to True if you want to use the online store
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:470086202700:feature-group/fraud-feature-group',
 'ResponseMetadata': {'RequestId': '7827701e-86c3-48aa-b91b-cb61f2cb9d52',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7827701e-86c3-48aa-b91b-cb61f2cb9d52',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '96',
   'date': 'Mon, 08 Jan 2024 18:38:21 GMT'},
  'RetryAttempts': 0}}

In [19]:
sagemaker_client.list_feature_groups() 

{'FeatureGroupSummaries': [{'FeatureGroupName': 'fraud-feature-group',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:470086202700:feature-group/fraud-feature-group',
   'CreationTime': datetime.datetime(2024, 1, 8, 18, 38, 22, 77000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'},
  {'FeatureGroupName': 'anamoly-feature-group',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:470086202700:feature-group/anamoly-feature-group',
   'CreationTime': datetime.datetime(2024, 1, 8, 16, 21, 9, 456000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}}],
 'ResponseMetadata': {'RequestId': 'e3083029-8c01-4d17-aa4d-4e34602cdfa4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e3083029-8c01-4d17-aa4d-4e34602cdfa4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '475',
   'date': 'Mon, 08 Jan 2024 18:38:24 GMT'},
  'RetryAttempts': 0}}

In [20]:
feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:470086202700:feature-group/fraud-feature-group',
 'FeatureGroupName': 'fraud-feature-group',
 'RecordIdentifierFeatureName': 'index',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'index', 'FeatureType': 'Integral'},
  {'FeatureName': 'Time', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V1', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V2', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V3', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V4', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V5', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V6', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V7', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V8', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V9', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V10', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V11', 'FeatureType': 'Fractional'},
  {'FeatureName': 'V12', 'FeatureType': 'Fraction

## Ingest Data

In [21]:
feature_group.ingest(data_frame=data, wait=True)

IngestionManagerPandas(feature_group_name='fraud-feature-group', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f02f3022650>, sagemaker_session=<sagemaker.session.Session object at 0x7f02fae7fb80>, max_workers=1, max_processes=1, profile_name=None, _async_result=None, _processing_pool=None, _failed_indices=[])

In [22]:
## Use amazon glue to find which tables are present you can use this table name for the next step/notebook
# Initialize the Glue client
glue_client = boto3.client('glue')

# Define the database name
database_name = 'sagemaker_featurestore'

# Get the list of tables
try:
    response = glue_client.get_tables(DatabaseName=database_name)
    tables = response['TableList']
    
    # Print table names and creation or update times
    for table in tables:
        print(f"Table Name: {table['Name']}")
        print(f"Creation Time: {table['CreateTime']}")
        print(f"Last Updated Time: {table['UpdateTime']}\n")
except Exception as e:
    print(f"An error occurred: {e}")


Table Name: anamoly_feature_group_1704730025
Creation Time: 2024-01-08 16:07:21+00:00
Last Updated Time: 2024-01-08 16:07:21+00:00

Table Name: anamoly_feature_group_1704730869
Creation Time: 2024-01-08 16:21:25+00:00
Last Updated Time: 2024-01-08 16:21:25+00:00

Table Name: fraud_feature_group_1704739102
Creation Time: 2024-01-08 18:38:39+00:00
Last Updated Time: 2024-01-08 18:38:39+00:00

